# Does using prior SQL queries help? 
In this workbook, we evalaute the effect of using "prior sql logs" in LLM prompts. For that, we will amend each table schema with the prior queries run on them. We need remove the query that is tested from the so called log. 


In [1]:
import pandas as pd
import tensorflow_hub as hub
import numpy as np
import altair as alt
from openai import OpenAI
from sql_metadata import Parser
import os
from IPython.display import display, Markdown
from collections import defaultdict 

In [26]:
df_q = pd.read_json('data/queries.json')

In [27]:
df_q

,question,sql,original_sql,tables_referred
0,How many organisation units in human resources...,SELECT COUNT(*) FROM (SELECT hr_org_unit_key F...,SELECT count(*) from (select hr_org_unit_key f...,"[HR_ORG_UNIT_NEW, HR_ORG_UNIT]"
1,How many organisation units in human resources...,SELECT COUNT(*) FROM (SELECT hr_org_unit_key F...,SELECT count(*) from (select hr_org_unit_key f...,"[HR_ORG_UNIT_NEW, HR_ORG_UNIT]"
2,Count the number of courses offered by each de...,"SELECT department_id, COUNT(*) AS course_count...","SELECT department_id, COUNT(*) AS course_count...",[CIS_COURSE_CATALOG]
3,"What are the past, upcoming, and current courses?","SELECT DISTINCT subject_title, effective_term_...","select distinct subject_title, effective_term_...","[CIS_COURSE_CATALOG, ACADEMIC_TERMS]"
4,Which faculty has had the most location changes?,SELECT MAX(cnt) FROM (SELECT COUNT(*) AS cnt F...,select max(cnt) from (select count(*) as cnt f...,[FCLT_BUILDING_ADDRESS_HIST]
5,What is the average number of courses a studen...,SELECT AVG(cnt) FROM (SELECT COUNT(*) AS cnt F...,select avg(cnt) from (select count(*) as cnt f...,"[MIT_STUDENT_DIRECTORY, SUBJECT_ENROLLABLE]"
6,Which students are employed by mit?,SELECT DISTINCT empl.full_name FROM mit_studen...,select distinct empl.full_name from mit_studen...,"[MIT_STUDENT_DIRECTORY, EMPLOYEE_DIRECTORY]"
7,What is the shortest course available this sem...,"SELECT DISTINCT subject_title, term_end_date -...","select distinct subject_title, term_end_date-t...","[ACADEMIC_TERMS, CIS_COURSE_CATALOG]"
8,Show buildings and their address.,"SELECT fb.building_name, fba.* FROM fac_buildi...","SELECT fb.building_name, fba.* FROM fac_buildi...","[FAC_BUILDING, FAC_BUILDING_ADDRESS]"
9,How many students do we have?,SELECT COUNT(*) FROM mit_student_directory;,select count(*) from mit_student_directory;,[MIT_STUDENT_DIRECTORY]


In [28]:
tables = []
for q in df_q['sql']: 
    t = Parser(q).tables
    tables.append([v.upper() for v in t])
df_q['tables'] = tables

In [29]:
unique_tables = set()
for  l in tables: 
    for t in l:
        unique_tables.add(t.upper())

In [30]:
len(unique_tables)

31

In [31]:
unique_tables

{'ACADEMIC_TERMS',
 'ACADEMIC_TERMS_ALL',
 'CIS_COURSE_CATALOG',
 'COURSE_CATALOG_SUBJECT_OFFERED',
 'EMPLOYEE_DIRECTORY',
 'FAC_BUILDING',
 'FAC_BUILDING_ADDRESS',
 'FCLT_BUILDING',
 'FCLT_BUILDING_ADDRESS_HIST',
 'FCLT_ROOMS',
 'HR_ORG_UNIT',
 'HR_ORG_UNIT_NEW',
 'IAP_SUBJECT_CATEGORY',
 'IAP_SUBJECT_DETAIL',
 'IAP_SUBJECT_PERSON',
 'IAP_SUBJECT_SESSION',
 'LIBRARY_MATERIAL_STATUS',
 'MIT_STUDENT_DIRECTORY',
 'SE_PERSON',
 'SPACE_UNIT',
 'STUDENT_DEPARTMENT',
 'SUBJECT_ENROLLABLE',
 'SUBJECT_OFFERED',
 'SUBJECT_SUMMARY',
 'TIP_DETAIL',
 'TIP_MATERIAL',
 'TIP_MATERIAL_STATUS',
 'TIP_SUBJECT_OFFERED',
 'TOP_LEVEL_DOMAIN',
 'ZIP_CANADA',
 'ZIP_USA'}

In [32]:
df_q

,question,sql,original_sql,tables_referred,tables
0,How many organisation units in human resources...,SELECT COUNT(*) FROM (SELECT hr_org_unit_key F...,SELECT count(*) from (select hr_org_unit_key f...,"[HR_ORG_UNIT_NEW, HR_ORG_UNIT]","[HR_ORG_UNIT_NEW, HR_ORG_UNIT]"
1,How many organisation units in human resources...,SELECT COUNT(*) FROM (SELECT hr_org_unit_key F...,SELECT count(*) from (select hr_org_unit_key f...,"[HR_ORG_UNIT_NEW, HR_ORG_UNIT]","[HR_ORG_UNIT_NEW, HR_ORG_UNIT]"
2,Count the number of courses offered by each de...,"SELECT department_id, COUNT(*) AS course_count...","SELECT department_id, COUNT(*) AS course_count...",[CIS_COURSE_CATALOG],[CIS_COURSE_CATALOG]
3,"What are the past, upcoming, and current courses?","SELECT DISTINCT subject_title, effective_term_...","select distinct subject_title, effective_term_...","[CIS_COURSE_CATALOG, ACADEMIC_TERMS]","[CIS_COURSE_CATALOG, ACADEMIC_TERMS]"
4,Which faculty has had the most location changes?,SELECT MAX(cnt) FROM (SELECT COUNT(*) AS cnt F...,select max(cnt) from (select count(*) as cnt f...,[FCLT_BUILDING_ADDRESS_HIST],[FCLT_BUILDING_ADDRESS_HIST]
5,What is the average number of courses a studen...,SELECT AVG(cnt) FROM (SELECT COUNT(*) AS cnt F...,select avg(cnt) from (select count(*) as cnt f...,"[MIT_STUDENT_DIRECTORY, SUBJECT_ENROLLABLE]","[MIT_STUDENT_DIRECTORY, SUBJECT_ENROLLABLE]"
6,Which students are employed by mit?,SELECT DISTINCT empl.full_name FROM mit_studen...,select distinct empl.full_name from mit_studen...,"[MIT_STUDENT_DIRECTORY, EMPLOYEE_DIRECTORY]","[MIT_STUDENT_DIRECTORY, EMPLOYEE_DIRECTORY]"
7,What is the shortest course available this sem...,"SELECT DISTINCT subject_title, term_end_date -...","select distinct subject_title, term_end_date-t...","[ACADEMIC_TERMS, CIS_COURSE_CATALOG]","[ACADEMIC_TERMS, CIS_COURSE_CATALOG]"
8,Show buildings and their address.,"SELECT fb.building_name, fba.* FROM fac_buildi...","SELECT fb.building_name, fba.* FROM fac_buildi...","[FAC_BUILDING, FAC_BUILDING_ADDRESS]","[FAC_BUILDING, FAC_BUILDING_ADDRESS]"
9,How many students do we have?,SELECT COUNT(*) FROM mit_student_directory;,select count(*) from mit_student_directory;,[MIT_STUDENT_DIRECTORY],[MIT_STUDENT_DIRECTORY]


In [33]:
df_x = pd.read_json('data/queries.json')

In [34]:
df_x['tables_referred'] 

0                        [HR_ORG_UNIT_NEW, HR_ORG_UNIT]
1                        [HR_ORG_UNIT_NEW, HR_ORG_UNIT]
2                                  [CIS_COURSE_CATALOG]
3                  [CIS_COURSE_CATALOG, ACADEMIC_TERMS]
4                          [FCLT_BUILDING_ADDRESS_HIST]
5           [MIT_STUDENT_DIRECTORY, SUBJECT_ENROLLABLE]
6           [MIT_STUDENT_DIRECTORY, EMPLOYEE_DIRECTORY]
7                  [ACADEMIC_TERMS, CIS_COURSE_CATALOG]
8                  [FAC_BUILDING, FAC_BUILDING_ADDRESS]
9                               [MIT_STUDENT_DIRECTORY]
10       [LIBRARY_MATERIAL_STATUS, TIP_MATERIAL_STATUS]
11                                [ZIP_USA, ZIP_CANADA]
12                                    [SUBJECT_OFFERED]
13                          [FCLT_ROOMS, FCLT_BUILDING]
14              [SPACE_UNIT, FCLT_ROOMS, FCLT_BUILDING]
15                                 [CIS_COURSE_CATALOG]
16                                 [CIS_COURSE_CATALOG]
17                                 [EMPLOYEE_DIR

In [23]:
unique_tables

{'ACADEMIC_TERMS',
 'ACADEMIC_TERMS_ALL',
 'CIS_COURSE_CATALOG',
 'COURSE_CATALOG_SUBJECT_OFFERED',
 'EMPLOYEE_DIRECTORY',
 'FAC_BUILDING',
 'FAC_BUILDING_ADDRESS',
 'FCLT_BUILDING',
 'FCLT_BUILDING_ADDRESS_HIST',
 'FCLT_ROOMS',
 'HR_ORG_UNIT',
 'HR_ORG_UNIT_NEW',
 'IAP_SUBJECT_CATEGORY',
 'IAP_SUBJECT_DETAIL',
 'IAP_SUBJECT_PERSON',
 'IAP_SUBJECT_SESSION',
 'LIBRARY_MATERIAL_STATUS',
 'MIT_STUDENT_DIRECTORY',
 'SE_PERSON',
 'SPACE_UNIT',
 'STUDENT_DEPARTMENT',
 'SUBJECT_ENROLLABLE',
 'SUBJECT_OFFERED',
 'SUBJECT_SUMMARY',
 'TIP_DETAIL',
 'TIP_MATERIAL',
 'TIP_MATERIAL_STATUS',
 'TIP_SUBJECT_OFFERED',
 'TOP_LEVEL_DOMAIN',
 'ZIP_CANADA',
 'ZIP_USA'}

In [35]:
unique_tables

{'ACADEMIC_TERMS',
 'ACADEMIC_TERMS_ALL',
 'CIS_COURSE_CATALOG',
 'COURSE_CATALOG_SUBJECT_OFFERED',
 'EMPLOYEE_DIRECTORY',
 'FAC_BUILDING',
 'FAC_BUILDING_ADDRESS',
 'FCLT_BUILDING',
 'FCLT_BUILDING_ADDRESS_HIST',
 'FCLT_ROOMS',
 'HR_ORG_UNIT',
 'HR_ORG_UNIT_NEW',
 'IAP_SUBJECT_CATEGORY',
 'IAP_SUBJECT_DETAIL',
 'IAP_SUBJECT_PERSON',
 'IAP_SUBJECT_SESSION',
 'LIBRARY_MATERIAL_STATUS',
 'MIT_STUDENT_DIRECTORY',
 'SE_PERSON',
 'SPACE_UNIT',
 'STUDENT_DEPARTMENT',
 'SUBJECT_ENROLLABLE',
 'SUBJECT_OFFERED',
 'SUBJECT_SUMMARY',
 'TIP_DETAIL',
 'TIP_MATERIAL',
 'TIP_MATERIAL_STATUS',
 'TIP_SUBJECT_OFFERED',
 'TOP_LEVEL_DOMAIN',
 'ZIP_CANADA',
 'ZIP_USA'}

In [36]:
len(unique_tables)

31

## Map tables to queries 

In [42]:
queries = []
for t in unique_tables:
    q = set()
    for i, T in enumerate(df_q['tables_referred']): 
        if t in T: 
            q.add(i) 
    queries.append(list(q))

In [43]:
queries

[[3, 7],
 [13, 14],
 [32],
 [34, 19],
 [13, 14],
 [2, 3, 7, 15, 16],
 [35, 5, 6, 9, 20],
 [32],
 [10],
 [31],
 [11, 22],
 [0, 1],
 [31],
 [32, 34, 29],
 [36],
 [11, 22],
 [0, 1],
 [8],
 [4],
 [12, 30],
 [27, 29],
 [28],
 [33, 31],
 [10],
 [14],
 [36, 6, 17, 18, 23, 25],
 [21],
 [24, 26],
 [19, 5],
 [8],
 [32]]

In [44]:
df_t = pd.DataFrame(columns=['table', 'log'])

In [46]:
df_t['table'] = list(unique_tables)

In [48]:
df_t['log'] = queries

In [49]:
df_t

,table,log
0,ACADEMIC_TERMS,"[3, 7]"
1,FCLT_ROOMS,"[13, 14]"
2,IAP_SUBJECT_SESSION,[32]
3,SUBJECT_SUMMARY,"[34, 19]"
4,FCLT_BUILDING,"[13, 14]"
5,CIS_COURSE_CATALOG,"[2, 3, 7, 15, 16]"
6,MIT_STUDENT_DIRECTORY,"[35, 5, 6, 9, 20]"
7,IAP_SUBJECT_DETAIL,[32]
8,TIP_MATERIAL_STATUS,[10]
9,TIP_DETAIL,[31]


# Give ChatGPT the "database" schema as context and elicit its answer

## Build the schema prompt 

In [122]:
def table_to_sql_create(t, l): 
    schema_file='data/views/schema_as_table/' + t + '.csv'
    p = None
    if os.path.isfile(schema_file):
        df_t = pd.read_csv(schema_file)
        p = "CREATE TABLE " + t + "(\n"  
        p = p + '\n'.join(list(df_t[['COLUMN_NAME', 'DATA_TYPE']].apply(lambda row: ' '.join(row.astype(str)), axis=1)))
        p = p + ')\n'
        if l:
            p = p + "\n ** PREVIOUS SQL QUERIES THAT REFERRED TO THE TABLE {0} **\n\n".format(t)
            for i in l: 
                # p = p + "QUESTION: {0} \n".format(df_q['question'][i])
                # p = p + "SQL ANSWER: {0} \n\n".format(df_q['sql'][i])
                p = p + "SQL: {0} \n\n".format(df_q['sql'][i])
    return p

In [123]:
def create_prompt(curr_query): 
    prompt = "*** ALL THE TABLES IN THE DATABASE and QUERY LOGS *** \n\n"
    cnt = 0 
    exclude = [curr_query]
    for i, t in enumerate(unique_tables):
        l = set(queries[i]).difference(exclude)
        p = table_to_sql_create(t, list(l))
        if not p:
            print(t)
        else: 
            cnt = cnt + 1
            prompt = prompt + p
            prompt = prompt + "\n\n"
    return prompt 

In [124]:
key = os.environ['OPENAI_API_KEY']

In [125]:
len(unique_tables)

31

In [126]:
client = OpenAI(api_key=key)

In [127]:
gpt_sql_log = []
for i, q in enumerate(df_q['question']): 
    prompt = create_prompt(i)
    response = client.chat.completions.create(
      model="gpt-4",
      messages = [
          {
          "role": "system",
          "content": "Your job is to write SQL queries that answer a user's question using the tables in the MIT Data Warehouse. \
          The MIT Data Warehouse is a central data source that combines data from various administrative systems at MIT, containing \
          information about students, faculty, and personnel. The schemas of the all the tables that you might need \
          for answering the user questions are given below. There are 31 tables in the database. \
          For each table, the schema of the table and the previous queries that referred to the table are also given. \
          \
          Reply with only the answer in SQL and include no linebreaks, newlines, escape characters or other commentary.\n\n" + prompt 
          # prompt 
          # Provide your answer in JSON form."
          # Reply with only the answer in JSON form and include no other commentary"
        },
          {"role":"user", 
           "content":q}]
          )
    gpt_sql_log.append(response.choices[0].message.content)

In [128]:
df_q['gpt_sql_log'] = gpt_sql_log

In [133]:
gpt_sql_log

['SELECT COUNT(*) FROM HR_ORG_UNIT_NEW MINUS SELECT COUNT(*) FROM HR_ORG_UNIT;',
 'SELECT COUNT(*) FROM HR_ORG_UNIT_NEW WHERE HR_ORG_UNIT_ID NOT IN (SELECT HR_ORG_UNIT_ID FROM HR_ORG_UNIT)',
 'SELECT DEPARTMENT_CODE, COUNT(*) FROM CIS_COURSE_CATALOG GROUP BY DEPARTMENT_CODE',
 "SELECT * FROM ACADEMIC_TERMS WHERE TERM_STATUS_INDICATOR IN ('P', 'F', 'C')",
 'SELECT RESPONSIBLE_FACULTY_NAME, COUNT(DISTINCT OFFICE_LOCATION) AS location_changes FROM EMPLOYEE_DIRECTORY GROUP BY RESPONSIBLE_FACULTY_NAME ORDER BY location_changes DESC LIMIT 1;',
 'SELECT AVG(NUM_ENROLLED_STUDENTS) FROM SUBJECT_OFFERED',
 "SELECT FULL_NAME FROM MIT_STUDENT_DIRECTORY WHERE DEPARTMENT = 'MIT'",
 "SELECT SUBJECT_TITLE, MIN(TERM_END_DATE - TERM_START_DATE) AS MINIMUM_DURATION FROM CIS_COURSE_CATALOG, ACADEMIC_TERMS WHERE CIS_COURSE_CATALOG.EFFECTIVE_TERM_CODE = ACADEMIC_TERMS.TERM_CODE AND ACADEMIC_TERMS.IS_CURRENT_TERM = 'Y' AND CIS_COURSE_CATALOG.IS_OFFERED_THIS_YEAR = 'Y' GROUP BY SUBJECT_TITLE ORDER BY MINIMUM_

In [134]:
df_q.to_csv("log-context-results.csv")

In [8]:
tables = []
for q in df_q['gpt_sql_log']: 
    t = Parser(q).tables
    tables.append([v.upper() for v in t])
df_q['gpt_sql_log_tables'] = tables


In [9]:
tables

[['HR_ORG_UNIT_NEW', 'HR_ORG_UNIT'],
 ['HR_ORG_UNIT_NEW', 'HR_ORG_UNIT'],
 ['CIS_COURSE_CATALOG'],
 ['ACADEMIC_TERMS'],
 ['EMPLOYEE_DIRECTORY'],
 ['SUBJECT_OFFERED'],
 ['MIT_STUDENT_DIRECTORY'],
 ['CIS_COURSE_CATALOG', 'ACADEMIC_TERMS'],
 ['FCLT_BUILDING', 'FAC_BUILDING_ADDRESS'],
 ['MIT_STUDENT_DIRECTORY'],
 ['TIP_MATERIAL_STATUS'],
 ['ZIP_USA', 'ZIP_CANADA'],
 ['CIS_COURSE_CATALOG'],
 ['FCLT_ROOMS'],
 ['SPACE_UNIT'],
 ['CIS_COURSE_CATALOG', 'ACADEMIC_TERMS'],
 ['SUBJECT_OFFERED'],
 ['EMPLOYEE_DIRECTORY'],
 ['EMPLOYEE_DIRECTORY'],
 ['SUBJECT_ENROLLABLE'],
 ['MIT_STUDENT_DIRECTORY'],
 ['IAP_SUBJECT_CATEGORY'],
 ['ZIP_USA', 'ZIP_CANADA'],
 ['EMPLOYEE_DIRECTORY'],
 ['STUDENT_DEPARTMENT'],
 ['EMPLOYEE_DIRECTORY'],
 ['STUDENT_DEPARTMENT'],
 ['SUBJECT_SUMMARY'],
 ['TOP_LEVEL_DOMAIN'],
 ['ACADEMIC_TERMS', 'CIS_COURSE_CATALOG'],
 ['SUBJECT_SUMMARY'],
 ['TIP_MATERIAL', 'TIP_DETAIL'],
 ['IAP_SUBJECT_DETAIL', 'SUBJECT_OFFERED'],
 ['TIP_MATERIAL'],
 ['SUBJECT_ENROLLABLE'],
 ['MIT_STUDENT_DIRECTOR

In [2]:
df_q = pd.read_csv("log-context-results.csv")

In [18]:
df_q

,Unnamed: 0,question,sql,original_sql,tables_referred,tables,gpt_sql_log,gpt_sql_log_tables
0,0,How many organisation units in human resources...,SELECT COUNT(*) FROM (SELECT hr_org_unit_key F...,SELECT count(*) from (select hr_org_unit_key f...,"['HR_ORG_UNIT_NEW', 'HR_ORG_UNIT']","['HR_ORG_UNIT_NEW', 'HR_ORG_UNIT']",SELECT COUNT(*) FROM HR_ORG_UNIT_NEW MINUS SEL...,"[HR_ORG_UNIT_NEW, HR_ORG_UNIT]"
1,1,How many organisation units in human resources...,SELECT COUNT(*) FROM (SELECT hr_org_unit_key F...,SELECT count(*) from (select hr_org_unit_key f...,"['HR_ORG_UNIT_NEW', 'HR_ORG_UNIT']","['HR_ORG_UNIT_NEW', 'HR_ORG_UNIT']",SELECT COUNT(*) FROM HR_ORG_UNIT_NEW WHERE HR_...,"[HR_ORG_UNIT_NEW, HR_ORG_UNIT]"
2,2,Count the number of courses offered by each de...,"SELECT department_id, COUNT(*) AS course_count...","SELECT department_id, COUNT(*) AS course_count...",['CIS_COURSE_CATALOG'],['CIS_COURSE_CATALOG'],"SELECT DEPARTMENT_CODE, COUNT(*) FROM CIS_COUR...",[CIS_COURSE_CATALOG]
3,3,"What are the past, upcoming, and current courses?","SELECT DISTINCT subject_title, effective_term_...","select distinct subject_title, effective_term_...","['CIS_COURSE_CATALOG', 'ACADEMIC_TERMS']","['CIS_COURSE_CATALOG', 'ACADEMIC_TERMS']",SELECT * FROM ACADEMIC_TERMS WHERE TERM_STATUS...,[ACADEMIC_TERMS]
4,4,Which faculty has had the most location changes?,SELECT MAX(cnt) FROM (SELECT COUNT(*) AS cnt F...,select max(cnt) from (select count(*) as cnt f...,['FCLT_BUILDING_ADDRESS_HIST'],['FCLT_BUILDING_ADDRESS_HIST'],"SELECT RESPONSIBLE_FACULTY_NAME, COUNT(DISTINC...",[EMPLOYEE_DIRECTORY]
5,5,What is the average number of courses a studen...,SELECT AVG(cnt) FROM (SELECT COUNT(*) AS cnt F...,select avg(cnt) from (select count(*) as cnt f...,"['MIT_STUDENT_DIRECTORY', 'SUBJECT_ENROLLABLE']","['MIT_STUDENT_DIRECTORY', 'SUBJECT_ENROLLABLE']",SELECT AVG(NUM_ENROLLED_STUDENTS) FROM SUBJECT...,[SUBJECT_OFFERED]
6,6,Which students are employed by mit?,SELECT DISTINCT empl.full_name FROM mit_studen...,select distinct empl.full_name from mit_studen...,"['MIT_STUDENT_DIRECTORY', 'EMPLOYEE_DIRECTORY']","['MIT_STUDENT_DIRECTORY', 'EMPLOYEE_DIRECTORY']",SELECT FULL_NAME FROM MIT_STUDENT_DIRECTORY WH...,[MIT_STUDENT_DIRECTORY]
7,7,What is the shortest course available this sem...,"SELECT DISTINCT subject_title, term_end_date -...","select distinct subject_title, term_end_date-t...","['ACADEMIC_TERMS', 'CIS_COURSE_CATALOG']","['ACADEMIC_TERMS', 'CIS_COURSE_CATALOG']","SELECT SUBJECT_TITLE, MIN(TERM_END_DATE - TERM...","[CIS_COURSE_CATALOG, ACADEMIC_TERMS]"
8,8,Show buildings and their address.,"SELECT fb.building_name, fba.* FROM fac_buildi...","SELECT fb.building_name, fba.* FROM fac_buildi...","['FAC_BUILDING', 'FAC_BUILDING_ADDRESS']","['FAC_BUILDING', 'FAC_BUILDING_ADDRESS']","SELECT BUILDING_NAME_LONG, STREET_NUMBER, STRE...","[FCLT_BUILDING, FAC_BUILDING_ADDRESS]"
9,9,How many students do we have?,SELECT COUNT(*) FROM mit_student_directory;,select count(*) from mit_student_directory;,['MIT_STUDENT_DIRECTORY'],['MIT_STUDENT_DIRECTORY'],SELECT COUNT(*) FROM MIT_STUDENT_DIRECTORY,[MIT_STUDENT_DIRECTORY]


In [32]:
accuracy = [] 
for i, t in enumerate(df_q['tables']):
    accuracy.append(len(set(t).intersection(df_q['gpt_sql_log_tables'][i]))/len(t))

In [33]:
df_q['tables']

0                        [HR_ORG_UNIT_NEW, HR_ORG_UNIT]
1                        [HR_ORG_UNIT_NEW, HR_ORG_UNIT]
2                                  [CIS_COURSE_CATALOG]
3                  [CIS_COURSE_CATALOG, ACADEMIC_TERMS]
4                          [FCLT_BUILDING_ADDRESS_HIST]
5           [MIT_STUDENT_DIRECTORY, SUBJECT_ENROLLABLE]
6           [MIT_STUDENT_DIRECTORY, EMPLOYEE_DIRECTORY]
7                  [ACADEMIC_TERMS, CIS_COURSE_CATALOG]
8                  [FAC_BUILDING, FAC_BUILDING_ADDRESS]
9                               [MIT_STUDENT_DIRECTORY]
10       [LIBRARY_MATERIAL_STATUS, TIP_MATERIAL_STATUS]
11                                [ZIP_USA, ZIP_CANADA]
12                                    [SUBJECT_OFFERED]
13                          [FCLT_ROOMS, FCLT_BUILDING]
14              [SPACE_UNIT, FCLT_ROOMS, FCLT_BUILDING]
15                                 [CIS_COURSE_CATALOG]
16                                 [CIS_COURSE_CATALOG]
17                                 [EMPLOYEE_DIR

In [48]:
accuracy

[1.0,
 1.0,
 1.0,
 0.5,
 0.0,
 0.0,
 0.5,
 1.0,
 0.5,
 1.0,
 0.5,
 1.0,
 0.0,
 0.5,
 0.3333333333333333,
 1.0,
 0.0,
 1.0,
 1.0,
 0.5,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.6666666666666666,
 0.25,
 1.0,
 0.0,
 1.0,
 0.5]

In [62]:
list(df_q['sql'][np.array(accuracy)==0.5])

["SELECT DISTINCT subject_title, effective_term_code, CASE WHEN term_start_date > SYSDATE THEN 'Upcoming' WHEN term_start_date <= SYSDATE AND term_end_date > SYSDATE THEN 'Current' ELSE 'Past' END AS term_status FROM cis_course_catalog, academic_terms WHERE effective_term_code = term_code;",
 'SELECT DISTINCT empl.full_name FROM mit_student_directory mit, employee_directory empl WHERE empl.full_name = mit.full_name;',
 'SELECT fb.building_name, fba.* FROM fac_building fb INNER JOIN fac_building_address fba ON fb.FAC_BUILDING_KEY = fba.building_key;',
 'SELECT DISTINCT library_material_status FROM library_material_status UNION SELECT DISTINCT tip_material_status FROM tip_material_status;',
 'SELECT DISTINCT BUILDING_NAME_LONG, COUNT(*) OVER (PARTITION BY BUILDING_NAME_LONG) FROM fclt_rooms, fclt_building WHERE fclt_rooms.fclt_building_key = fclt_building.fclt_building_key;',
 "SELECT DISTINCT se.subject_id, se.subject_title FROM subject_summary su, subject_enrollable se WHERE se.subject

In [63]:
list(df_q['question'][np.array(accuracy)==0.5])

['What are the past, upcoming, and current courses?',
 'Which students are employed by mit?',
 'Show buildings and their address.',
 'Show all statuses for materials?',
 'Count the number of room in a building.',
 'Show the name for subjects which are enrollable for Computer Science Student.',
 'Which department has the most kinds of payroll ranks?']

In [64]:
list(df_q['tables'][np.array(accuracy)==0.5])

[['CIS_COURSE_CATALOG', 'ACADEMIC_TERMS'],
 ['MIT_STUDENT_DIRECTORY', 'EMPLOYEE_DIRECTORY'],
 ['FAC_BUILDING', 'FAC_BUILDING_ADDRESS'],
 ['LIBRARY_MATERIAL_STATUS', 'TIP_MATERIAL_STATUS'],
 ['FCLT_ROOMS', 'FCLT_BUILDING'],
 ['SUBJECT_SUMMARY', 'SUBJECT_ENROLLABLE'],
 ['SE_PERSON', 'EMPLOYEE_DIRECTORY']]

In [65]:
list(df_q['gpt_sql_log_tables'][np.array(accuracy)==0.5])

[['ACADEMIC_TERMS'],
 ['MIT_STUDENT_DIRECTORY'],
 ['FCLT_BUILDING', 'FAC_BUILDING_ADDRESS'],
 ['TIP_MATERIAL_STATUS'],
 ['FCLT_ROOMS'],
 ['SUBJECT_ENROLLABLE'],
 ['SE_PERSON']]

In [53]:
list(df_q['question'][np.array(accuracy)==0.5])

['What are the past, upcoming, and current courses?',
 'Which students are employed by mit?',
 'Show buildings and their address.',
 'Show all statuses for materials?',
 'Count the number of room in a building.',
 'Show the name for subjects which are enrollable for Computer Science Student.',
 'Which department has the most kinds of payroll ranks?']

In [55]:
list(df_q['sql'][np.array(accuracy)==0.5])

["SELECT DISTINCT subject_title, effective_term_code, CASE WHEN term_start_date > SYSDATE THEN 'Upcoming' WHEN term_start_date <= SYSDATE AND term_end_date > SYSDATE THEN 'Current' ELSE 'Past' END AS term_status FROM cis_course_catalog, academic_terms WHERE effective_term_code = term_code;",
 'SELECT DISTINCT empl.full_name FROM mit_student_directory mit, employee_directory empl WHERE empl.full_name = mit.full_name;',
 'SELECT fb.building_name, fba.* FROM fac_building fb INNER JOIN fac_building_address fba ON fb.FAC_BUILDING_KEY = fba.building_key;',
 'SELECT DISTINCT library_material_status FROM library_material_status UNION SELECT DISTINCT tip_material_status FROM tip_material_status;',
 'SELECT DISTINCT BUILDING_NAME_LONG, COUNT(*) OVER (PARTITION BY BUILDING_NAME_LONG) FROM fclt_rooms, fclt_building WHERE fclt_rooms.fclt_building_key = fclt_building.fclt_building_key;',
 "SELECT DISTINCT se.subject_id, se.subject_title FROM subject_summary su, subject_enrollable se WHERE se.subject

In [57]:
list(df_q['gpt_sql_log'][np.array(accuracy)==0.5])

["SELECT * FROM ACADEMIC_TERMS WHERE TERM_STATUS_INDICATOR IN ('P', 'F', 'C')",
 "SELECT FULL_NAME FROM MIT_STUDENT_DIRECTORY WHERE DEPARTMENT = 'MIT'",
 'SELECT BUILDING_NAME_LONG, STREET_NUMBER, STREET_NAME, CITY, STATE, POSTAL_CODE FROM FCLT_BUILDING JOIN FAC_BUILDING_ADDRESS ON FCLT_BUILDING.FCLT_BUILDING_KEY = FAC_BUILDING_ADDRESS.BUILDING_KEY',
 'SELECT DISTINCT TIP_MATERIAL_STATUS FROM TIP_MATERIAL_STATUS;',
 'SELECT BUILDING_ROOM, COUNT(*) AS ROOM_COUNT FROM FCLT_ROOMS GROUP BY BUILDING_ROOM',
 "SELECT DISTINCT SUBJECT_TITLE FROM SUBJECT_ENROLLABLE WHERE OFFER_DEPT_CODE='Computer Science'",
 'SELECT DEPARTMENT, COUNT(DISTINCT PAYROLL_RANK) AS RANK_COUNT FROM SE_PERSON GROUP BY DEPARTMENT ORDER BY RANK_COUNT DESC LIMIT 1']

In [158]:
np.mean(np.array(accuracy))

0.6418918918918919

In [169]:
tp = 0 
fp = 0 
fn = 0 
accurate = 0 
for i, t in enumerate(df_q['tables']):
    tp = tp + len(set(t).intersection(tables[i]))
    fn = fn + len(set(t).difference(tables[i]))
    fp = fp + len(set(tables[i]).difference(t))
    accurate = accurate + (1 if set(t) == set(tables[i]) else 0) 
precision = tp / (tp + fp) 
recall = tp / (tp + fn)
accuracy = accurate / len(tables) 
print("precision:{0}".format(precision))
print("recall:{0}".format(recall))
print("f1:{0}".format(2*precision*recall/(precision + recall)))
print("accuracy:{0}".format(accuracy))

precision:0.7446808510638298
recall:0.5932203389830508
f1:0.6603773584905661
accuracy:0.4864864864864865


In [25]:
df_x = pd.read_json('data/queries.json')

In [27]:
df_q['tables_referred']=df_x['tables_referred']

In [31]:
df_q['tables']=df_q['tables_referred']